1. Load the dataset provided. **(3 pts)**
   - Ensure all columns have the correct data type, e.g., `float` or `int` for numeric quantities, `object` for categorical variables, `datetime` for dates, etc.
   - Remove columns where all values are missing or equal to zero.
   - Describe the basic statistics (mean, minimum, maximum, standard deviation, etc.) of the numerical variables.

2. Perform all data pre-processing steps needed for the model (i.e., integration, cleaning, reduction, and transformation). **(7 pts)**  
   For each pre-processing step that you apply, dedicate at least one markdown cell next to the code to explain:
   - the step you are applying
   - why you need to do it
   - what is the result  

   **Note:** In some scenarios, it is acceptable to repeat steps; however, make sure your code is logical and efficient.

3. Organize your time-dependent data. In a separate markdown cell, summarize how you organize your time-dependent data. Motivate your strategy. You can include a figure if needed. **(4 pts)**
   - **Hint:** you can include a figure in markdown syntax as follows:  

4. Prepare your data for model selection and evaluation. **(4 pts)**  
   What is the experimental setup that you use to build the model? Please provide a figure that summarizes your setup where you clearly mark:
   - how do you split the available data for training, validation, or testing
   - what type of validation strategy do you use (e.g., hold out, cross-validation)

5. Choose a recurrent neural network architecture to build the prediction model (e.g., LSTM, GRU, etc.) and implement your solution. Motivate which hyperparameters you select or optimize. **(5 pts)**

6. Create plots of training and validation losses. Describe the trends you notice and whether they indicate underfitting or overfitting behaviors. **(2 pts)**

7. Evaluate the performance of the model. Describe which measures you use and discuss the results of the evaluation. **(3 pts)**

8. Create a plot contrasting the predictions of the model and the ground truth values from the test set. **(2 pts)**